In [8]:
# Calc. Spinpolalization from smoothing and interpolated DOS

In [5]:
import os
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
from scipy.ndimage import gaussian_filter1d

In [7]:
TDOS_list = sorted(os.listdir("../data/TDOS/"))
# TDOS_list

In [5]:
composition = np.arange(1, 100, 1)
# composition

In [6]:
def DOS_ip(filepath, sigma=0.1, ip_range=(-4, 6, 0.05)):
    df = pd.read_csv(filepath, header=None, skiprows=2, names=["energy", "TDOS_up", "TDOS_down"])
    # ガウシアンフィルタを適用してスムージングする
    df["TDOS_up_s"] = gaussian_filter1d(df["TDOS_up"], sigma=sigma)
    df["TDOS_down_s"] = gaussian_filter1d(df["TDOS_down"], sigma=sigma)
    cs_up = CubicSpline(df["energy"], df["TDOS_up_s"])
    cs_down = CubicSpline(df["energy"], df["TDOS_down_s"])
    x_ip = np.arange(ip_range[0],ip_range[1],ip_range[2])
    up_ip = cs_up(x_ip)
    down_ip = cs_down(x_ip)
    df_ip = pd.DataFrame({"energy_ip":x_ip, "TDOS_up_ip":up_ip, "TDOS_down_ip":down_ip})

    return df, df_ip

In [7]:
df, df_ip = DOS_ip(f"./TDOS/{TDOS_list[0]}", sigma=1)

In [ ]:
plt.plot(df["energy"], df["TDOS_up"], marker='o', linestyle="None", label='Data points', markersize=3)  # サンプルデータをプロット
plt.plot(df_ip["energy_ip"], df_ip["TDOS_up_ip"], marker='^', linestyle="-", markersize=3, label='Smoothed Data points')
plt.xlim(-1, 1)
plt.legend()
plt.show()

In [24]:
def calc_spin_pol(df_ip):
    selected_values = df_ip.loc[round(df_ip["energy_ip"], 2) == 0, ["TDOS_up_ip", "TDOS_down_ip"]]
    sp_pol = (selected_values.values[0][0]+selected_values.values[0][1])/(selected_values.values[0][0]-selected_values.values[0][1])
    return sp_pol

In [31]:
sp_pol_list=[]
for DOSfile in TDOS_list:
    df, df_ip = DOS_ip(f"./TDOS/{DOSfile}", sigma=1)
    # print(DOSfile)
    sp_pol_list.append(calc_spin_pol(df_ip))
# sp_pol_list

In [ ]:
plt.plot(composition, sp_pol_list)

In [89]:
df_csv = pd.DataFrame(np.array([composition, sp_pol_list]).T, columns=["composition", "spin_polarization"])
df_csv.to_csv("spin_polarization.csv", index=None)